In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import re

In [2]:
#importing datafile and create dataframe data collection
df_che = pd.read_csv("CHE.3000.csv")
df = pd.DataFrame(columns = ["CHE_name",'CNES_name','Address',"Type",'Campus_Size','Housing','Student_Population','Sudent/Faculty_Ratio','Faculty_Population_FT', 'Faculty_Population_PT', 'Tuition_Fee_2016-2017',
 'Tuition_Fee_2017-2018','Tuition_Fee_2018-2019', 'Tuition_Fee_2019-2020', 'Total_Expenses_2016-2017', 'Total_Expenses_2017-2018', 'Total_Expenses_2018-2019', 'Total_Expenses_2019-2020', 
 'AVG_Tuition-Grad-2019-2020','AVG_Fee-Grad-2019-2020','Number-Institution_Grants/Scholarship', 'Percent_Institution_Grants/Scholarship', 'AVG_Institution_Grants/Scholarship_Received',
 'Total_Institution_Grants/Scholarship_Received', 'Number_All_Undergrad_Grant/Scholarship', 'Total_All_Undergrad_Grant/Scholarship', 'AVG_All_Undergrad_Grant/Scholarship', 'Number_Federal_Student_Loans', 'Total_Federal_Student_Loans',
 'AVG_Dederal_Student_Loans_Amount','AVG_Netprice_FT_BeginningStudent_2016-2017','AVG_Netprice_FT_BeginningStudent_2017-2017', 'AVG_Netprice_FT_BeginningStudent_2017-2018'])


In [ ]:
df_che["Name"] = df_che["Name"].str.replace("U.","University")

In [ ]:
df_che["Name"] = df_che["Name"].str.replace(",","-")
df_che["Name"] = df_che["Name"].str.replace("at ","-")
df_che["Name"] = df_che["Name"].str.replace(".","")
df_che["Name"] = df_che["Name"].str.replace("City University of New York","CUNY")

In [ ]:
def remove_parents(x):
    start = x.find("(")
    if start !=-1:
        result= x[:start]
    else:
        result = x
    return result

In [ ]:
df_che["Name"] = df_che["Name"].apply(remove_parents)

In [ ]:
df_che["Name"]

In [ ]:
df_che[df_che.Name.str.contains("U.")]

In [67]:
#Initiatiing webdriver to browse using Chrome in Incognito mode
path = '/Users/Petelindan/Downloads/chromedriver_win32/chromedriver'
chrome_options = Options()
chrome_options.add_argument(" — incognito")
browser = webdriver.Chrome(path, options=chrome_options)

#Open the data source using Chrome 
start_url='https://nces.ed.gov/collegenavigator/'
browser.get(start_url)
error_school = []
error_tuition = []
error_grad = []
#Locating search input element to enter school name

In [69]:
error_name1= []
for school,loc in error_school:
     #For each school, search for the name and navigate to the result page
    #Clearing previous selection and input
    start_url = 'https://nces.ed.gov/collegenavigator/'
    browser.get(start_url
               )
    keys = school
    browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName").clear()
    location = Select(browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_ucMapMain_lstState"))
    location.deselect_all()

    #input search keywords
    search_tab = browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName")
    search_tab.send_keys(keys)
    location.select_by_value(loc)
    browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_btnSearch").click()

    #Result selection page
    try:
        newURL = browser.current_url
        browser.get(newURL)
        browser.find_element_by_xpath("//tr[@class = 'resultsW']/td[2]/a[@href]").click()
    
        #Extracting data
        browser.find_element_by_xpath("//a[contains(text(),'Expand All')]").click()
        name = browser.find_element_by_xpath("//span[@class='headerlg']").text
        address = browser.find_element_by_xpath("//span[@style='position:relative']").text.split("\n")[1]
        school_type = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[3]").text
        size = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[5]//td[2]").text
        housing = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[6]//td[2]").text
        student_population = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[7]").text.split(":")[1].strip()
        student_faculty_ratio = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[8]//td[2]").text.replace("to",":")
        faculty_population_FT =  browser.find_element_by_xpath("//*[contains(text(), 'Total faculty')]/following-sibling::td[1]").text
        faculty_population_PT =  browser.find_element_by_xpath("//*[contains(text(), 'Total faculty')]/following-sibling::td[2]").text
        tuition_fee_2016_2017 =  browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[1]").text
        tuition_fee_2017_2018 =  browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[2]").text
        tuition_fee_2018_2019 = browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[3]").text
        tuition_fee_2019_2020 = browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[4]").text
        expense = browser.find_element_by_xpath("//div[@id = 'divctl00_cphCollegeNavBody_ucInstitutionMain_ctl00']").text.replace("\n",". ").split("TOTAL EXPENSES 2016-2017 2017-2018 2018-2019 2019-2020 % CHANGE 2018-2019 TO 2019-2020.")[1]
        if "On Campus" in expense:
            expense = expense.split("On Campus")[1].split(".")[0].strip().split(" ")
        else:
            expense = expense.split("Off Campus")[1].split(".")[0].strip().split(" ")
        total_expenses_2016_2017 = expense[0]
        total_expenses_2017_2018 = expense[1]
        total_expenses_2018_2019 = expense[2]
        total_expenses_2019_2020 = expense[3]
        if browser.find_elements_by_xpath("//*[contains(text(),'Average graduate student tuition and fees for academic year')]"):
            grad = browser.find_element_by_xpath("//div[@id = 'divctl00_cphCollegeNavBody_ucInstitutionMain_ctl00']").text.replace("\n",". ").split("AVERAGE GRADUATE STUDENT TUITION AND FEES FOR ACADEMIC YEAR 2019-2020")
            avg_tuition_grad_2019_2020 = grad[1].split(".")[1].strip().split(" ")[1]
            avg_fee_grad_2019_2020 = grad[1].split(".")[2].strip().split(" ")[1]
        else:
            avg_tuition_grad_2019_2020 = np.nan
            avg_fee_grad_2019_2020 = np. nan
            
        number_all_undergrad_grant_scholarship = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Grant or scholarship aid']/following-sibling::td[1]").text
        total_all_undergrad_grant_scholarship = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Grant or scholarship aid']/following-sibling::td[2]").text
        avg_all_undergrad_grant_scholarship = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Grant or scholarship aid']/following-sibling::td[3]").text
        number_federal_student_loans = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Federal student loans']/following-sibling::td[1]").text
        total_federal_student_loans = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Federal student loans']/following-sibling::td[2]").text
        avg_federal_student_loans = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Federal student loans']/following-sibling::td[3]").text
        avg_2016_2017_netprice_FT_BeginningStudent = browser.find_element_by_xpath("//td[contains(text(),'Average net price')]/following-sibling::td[1]").text
        avg_2017_2018_netprice_FT_BeginningStudent = browser.find_element_by_xpath("//td[contains(text(),'Average net price')]/following-sibling::td[2]").text
        avg_2018_2019_netprice_FT_BeginningStudent = browser.find_element_by_xpath("//td[contains(text(),'Average net price')]/following-sibling::td[3]").text
        
        school_data = {"CHE_name":school,"CNES_name":name,"Address":address,"Type":school_type,"Campus_Size":size,"Housing":housing,"Student_Population":student_population,
                          "Sudent/Faculty_Ratio":student_faculty_ratio,"Faculty_Population_FT":faculty_population_FT,"Faculty_Population_PT":faculty_population_PT,
                          "Tuition_Fee_2016-2017": tuition_fee_2016_2017,"Tuition_Fee_2017-2018":tuition_fee_2017_2018,"Tuition_Fee_2018-2019":tuition_fee_2018_2019,"Tuition_Fee_2019-2020":tuition_fee_2019_2020,
                          "Total_Expenses_2016-2017":total_expenses_2016_2017,"Total_Expenses_2017-2018":total_expenses_2017_2018,"Total_Expenses_2018-2019":total_expenses_2018_2019,
                          "Total_Expenses_2019-2020":total_expenses_2019_2020,"AVG_Tuition-Grad-2019-2020":avg_tuition_grad_2019_2020,"AVG_Fee-Grad-2019-2020":avg_fee_grad_2019_2020,
                        "Number_All_Undergrad_Grant/Scholarship":number_all_undergrad_grant_scholarship,
                          "Total_All_Undergrad_Grant/Scholarship":total_all_undergrad_grant_scholarship,"AVG_All_Undergrad_Grant/Scholarship":avg_all_undergrad_grant_scholarship,
                          "Number_Federal_Student_Loans":number_federal_student_loans,"Total_Federal_Student_Loans":total_federal_student_loans,"AVG_Dederal_Student_Loans_Amount":avg_federal_student_loans,
                          "AVG_Netprice_FT_BeginningStudent_2016-2017":avg_2016_2017_netprice_FT_BeginningStudent,"AVG_Netprice_FT_BeginningStudent_2017-2017":avg_2017_2018_netprice_FT_BeginningStudent,
                          "AVG_Netprice_FT_BeginningStudent_2017-2018":avg_2018_2019_netprice_FT_BeginningStudent}
        df = df.append(school_data,ignore_index=True)
    except:
        error_name1.append([school,loc])
        pass

In [50]:
# df.info()
error_name1

[['Academy for Nursing and Health Occup-ions', 'FL'],
 ['Adams St-e University', 'CO'],
 ['Akron Institute of Herzing University', 'OH'],
 ['Alabama A&M University', 'AL'],
 ['Alabama St-e University', 'AL'],
 ['Albany St-e University', 'GA'],
 ['Alcorn St-e University', 'MS'],
 ['Alfred St-e College', 'NY'],
 ['Allen College', 'IA'],
 ['Altierus Career College - Henderson ', 'NV'],
 ['Altierus Career University - Tampa ', 'FL'],
 ['American Academy of Dram-ic Arts - Los Angeles', 'CA'],
 ['American Academy of Dram-ic Arts - New York', 'NY'],
 ['American Conserv-ory The-er', 'CA'],
 ['American Intern-ional College', 'MA'],
 ['American Jewish University', 'CA'],
 ['American Musical and Dram-ic Academy', 'NY'],
 ['Angelo St-e University', 'TX'],
 ['Apex School of Theology', 'NC'],
 ['Appalachian St-e University', 'NC'],
 ['Arizona St-e University', 'AZ'],
 ['Arizona St-e University-Downtown Phoenix', 'AZ'],
 ['Arizona St-e University-Polytechnic', 'AZ'],
 ['Arizona St-e University-Skyson

In [59]:
df.to_csv("web_scrapping_3000_scrape.csv")

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2504 entries, 0 to 2503
Data columns (total 33 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   CHE_name                                       2504 non-null   object
 1   CNES_name                                      2504 non-null   object
 2   Address                                        2504 non-null   object
 3   Type                                           2504 non-null   object
 4   Campus_Size                                    2504 non-null   object
 5   Housing                                        2504 non-null   object
 6   Student_Population                             2504 non-null   object
 7   Sudent/Faculty_Ratio                           2504 non-null   object
 8   Faculty_Population_FT                          2504 non-null   object
 9   Faculty_Population_PT                          2504 non-null   

In [72]:
error_name1

[['Akron Institute of Herzing University', 'OH'],
 ['Alabama A&M University', 'AL'],
 ['Allen College', 'IA'],
 ['Altierus Career College -Henderson ', 'NV'],
 ['Altierus Career University -Tampa ', 'FL'],
 ['American Conservatory Theater', 'CA'],
 ['American Jewish University', 'CA'],
 ['Apex School of Theology', 'NC'],
 ['Arizona State University', 'AZ'],
 ['Auburn University -Montgomery', 'AL'],
 ['Averett University Nontraditional Programs', 'VA'],
 ['Baker College of Flint', 'MI'],
 ['Bakersfield College', 'CA'],
 ['Baptist Health College -Little Rock ', 'AR'],
 ['Baptist Missionary Association Theological Seminary', 'TX'],
 ["Bard College -Simon's Rock", 'MA'],
 ['Becker College -Worcester ', 'MA'],
 ['Belanger School of Nursing', 'NY'],
 ['Bethany College ', 'KS'],
 ['Bethel College ', 'IN'],
 ['Bethesda University of California', 'CA'],
 ['Bill and Sandra Pomeroy College of Nursing -Crouse Hospital', 'NY'],
 ['Bluefield College', 'VA'],
 ['Boston Baptist College', 'MA'],
 ['Bow

In [83]:
for i in error_name1:
    i[0]= i[0].replace("-","at ")
    i[0] = i[0].replace("State University of New York College","SUNY")

In [85]:
error_name2 = []
for school,loc in error_name1:
    try:
    #For each school, search for the name and navigate to the result page
    #Clearing previous selection and input
        keys = school
        browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName").clear()
        location = Select(browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_ucMapMain_lstState"))
        location.deselect_all()

        #input search keywords
        search_tab = browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName")
        search_tab.send_keys(keys)
        location.select_by_value(loc)
        browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_btnSearch").click()

        #Result selection page

        newURL = browser.current_url
        browser.get(newURL)
        #         browser.find_element_by_xpath("//a[@id='ctl00_cphCollegeNavBody_ucResultsMain_aSortName']").click()
        browser.find_element_by_xpath("//tr[@class = 'resultsW']/td[2]/a[@href]").click()
        #Extracting data
        browser.find_element_by_xpath("//a[contains(text(),'Expand All')]").click()
        name = browser.find_element_by_xpath("//span[@class='headerlg']").text
        address = browser.find_element_by_xpath("//span[@style='position:relative']").text.split("\n")[1]
        size = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[5]//td[2]").text
        housing = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[6]//td[2]").text
        student_population = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[7]").text.split(":")[1].strip()
        student_faculty_ratio = browser.find_element_by_xpath("//table[@class = 'layouttab']//tr[8]//td[2]").text.replace("to",":")
        faculty_population_FT =  browser.find_element_by_xpath("//*[contains(text(), 'Total faculty')]/following-sibling::td[1]").text
        faculty_population_PT =  browser.find_element_by_xpath("//*[contains(text(), 'Total faculty')]/following-sibling::td[2]").text
        tuition_fee_2016_2017 =  browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[1]").text
        tuition_fee_2017_2018 =  browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[2]").text
        tuition_fee_2018_2019 = browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[3]").text
        tuition_fee_2019_2020 = browser.find_element_by_xpath("//*[contains(text(),'Tuition and fees')]/following-sibling::td[4]").text
        expense = browser.find_element_by_xpath("//div[@id = 'divctl00_cphCollegeNavBody_ucInstitutionMain_ctl00']").text.replace("\n",". ").split("TOTAL EXPENSES 2016-2017 2017-2018 2018-2019 2019-2020 % CHANGE 2018-2019 TO 2019-2020.")[1]
        if "On Campus" in expense:
            expense = expense.split("On Campus")[1].split(".")[0].strip().split(" ")
        else:
            expense = expense.split("Off Campus")[1].split(".")[0].strip().split(" ")
        total_expenses_2016_2017 = expense[0]
        total_expenses_2017_2018 = expense[1]
        total_expenses_2018_2019 = expense[2]
        total_expenses_2019_2020 = expense[3]
        if browser.find_elements_by_xpath("//*[contains(text(),'Average graduate student tuition and fees for academic year')]"):
            grad = browser.find_element_by_xpath("//div[@id = 'divctl00_cphCollegeNavBody_ucInstitutionMain_ctl00']").text.replace("\n",". ").split("AVERAGE GRADUATE STUDENT TUITION AND FEES FOR ACADEMIC YEAR 2019-2020")
            avg_tuition_grad_2019_2020 = grad[1].split(".")[1].strip().split(" ")[1]
            avg_fee_grad_2019_2020 = grad[1].split(".")[2].strip().split(" ")[1]
        else:
            avg_tuition_grad_2019_2020 = np.nan
            avg_fee_grad_2019_2020 = np. nan
        number_institution_grants_scholarship =  browser.find_element_by_xpath("//*[text() = 'Institutional grants or scholarships']/following-sibling::td[1]").text
        percent_institution_grants_scholarship = browser.find_element_by_xpath("//*[text() = 'Institutional grants or scholarships']/following-sibling::td[2]").text
        total_institution_grants_scholarship = browser.find_element_by_xpath("//*[text() = 'Institutional grants or scholarships']/following-sibling::td[3]").text
        avg_institution_grands_scholarship_received = browser.find_element_by_xpath("//*[text() = 'Institutional grants or scholarships']/following-sibling::td[4]").text
        number_all_undergrad_grant_scholarship = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Grant or scholarship aid']/following-sibling::td[1]").text
        total_all_undergrad_grant_scholarship = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Grant or scholarship aid']/following-sibling::td[2]").text
        avg_all_undergrad_grant_scholarship = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Grant or scholarship aid']/following-sibling::td[3]").text
        number_federal_student_loans = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Federal student loans']/following-sibling::td[1]").text
        total_federal_student_loans = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Federal student loans']/following-sibling::td[2]").text
        avg_federal_student_loans = browser.find_element_by_xpath("//*[text() = 'All Undergraduate Students']/following-sibling::table[@class = 'tabular']//tbody//*[text()= 'Federal student loans']/following-sibling::td[3]").text
        avg_2016_2017_netprice_FT_BeginningStudent = browser.find_element_by_xpath("//td[contains(text(),'Average net price')]/following-sibling::td[1]").text
        avg_2017_2018_netprice_FT_BeginningStudent = browser.find_element_by_xpath("//td[contains(text(),'Average net price')]/following-sibling::td[2]").text
        avg_2018_2019_netprice_FT_BeginningStudent = browser.find_element_by_xpath("//td[contains(text(),'Average net price')]/following-sibling::td[3]").text

        school_data = {"Institution":name,"Address":address,"Campus_Size":size,"Housing":housing,"Student_Population":student_population,
                          "Sudent/Faculty_Ratio":student_faculty_ratio,"Faculty_Population_FT":faculty_population_FT,"Faculty_Population_PT":faculty_population_PT,
                          "Tuition_Fee_2016-2017": tuition_fee_2016_2017,"Tuition_Fee_2017-2018":tuition_fee_2017_2018,"Tuition_Fee_2018-2019":tuition_fee_2018_2019,"Tuition_Fee_2019-2020":tuition_fee_2019_2020,
                          "Total_Expenses_2016-2017":total_expenses_2016_2017,"Total_Expenses_2017-2018":total_expenses_2017_2018,"Total_Expenses_2018-2019":total_expenses_2018_2019,
                          "Total_Expenses_2019-2020":total_expenses_2019_2020,"AVG_Tuition-Grad-2019-2020":avg_tuition_grad_2019_2020,"AVG_Fee-Grad-2019-2020":avg_fee_grad_2019_2020,
                          "Number-Institution_Grants/Scholarship":number_institution_grants_scholarship,"Percent_Institution_Grants/Scholarship" : percent_institution_grants_scholarship,
                          "AVG_Institution_Grants/Scholarship_Received":avg_institution_grands_scholarship_received,"Total_Institution_Grants/Scholarship_Received": total_institution_grants_scholarship,
                        "Number_All_Undergrad_Grant/Scholarship":number_all_undergrad_grant_scholarship,
                          "Total_All_Undergrad_Grant/Scholarship":total_all_undergrad_grant_scholarship,"AVG_All_Undergrad_Grant/Scholarship":avg_all_undergrad_grant_scholarship,
                          "Number_Federal_Student_Loans":number_federal_student_loans,"Total_Federal_Student_Loans":total_federal_student_loans,"AVG_Dederal_Student_Loans_Amount":avg_federal_student_loans,
                          "AVG_Netprice_FT_BeginningStudent_2016-2017":avg_2016_2017_netprice_FT_BeginningStudent,"AVG_Netprice_FT_BeginningStudent_2017-2017":avg_2017_2018_netprice_FT_BeginningStudent,
                          "AVG_Netprice_FT_BeginningStudent_2017-2018":avg_2018_2019_netprice_FT_BeginningStudent}
        df = df.append(school_data,ignore_index=True)
    except:
        error_name2.append([school,loc])
        pass

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 34 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   CHE_name                                       2504 non-null   object
 1   CNES_name                                      2504 non-null   object
 2   Address                                        2560 non-null   object
 3   Type                                           2504 non-null   object
 4   Campus_Size                                    2560 non-null   object
 5   Housing                                        2560 non-null   object
 6   Student_Population                             2560 non-null   object
 7   Sudent/Faculty_Ratio                           2560 non-null   object
 8   Faculty_Population_FT                          2560 non-null   object
 9   Faculty_Population_PT                          2560 non-null   

In [88]:
# df.to_csv("web_scrapping_3000_scrape.csv")

In [33]:
df = pd.read_csv("web_scrapping_3000_scrape.csv")
df.head(20)

,Unnamed: 0,CHE_name,CNES_name,Address,Type,Campus_Size,Housing,Student_Population,Sudent/Faculty_Ratio,Faculty_Population_FT,Faculty_Population_PT,Tuition_Fee_2016-2017,Tuition_Fee_2017-2018,Tuition_Fee_2018-2019,Tuition_Fee_2019-2020,Total_Expenses_2016-2017,Total_Expenses_2017-2018,Total_Expenses_2018-2019,Total_Expenses_2019-2020,AVG_Tuition-Grad-2019-2020,AVG_Fee-Grad-2019-2020,Number-Institution_Grants/Scholarship,Percent_Institution_Grants/Scholarship,AVG_Institution_Grants/Scholarship_Received,Total_Institution_Grants/Scholarship_Received,Number_All_Undergrad_Grant/Scholarship,Total_All_Undergrad_Grant/Scholarship,AVG_All_Undergrad_Grant/Scholarship,Number_Federal_Student_Loans,Total_Federal_Student_Loans,AVG_Dederal_Student_Loans_Amount,AVG_Netprice_FT_BeginningStudent_2016-2017,AVG_Netprice_FT_BeginningStudent_2017-2017,AVG_Netprice_FT_BeginningStudent_2017-2018,Institution
0,0,Aaniiih Nakoda College,Aaniiih Nakoda College,"269 Blackfeet Avenue Agency, Harlem, Montana 5...","Type: 2-year, Public",Rural: Remote,No,131 (all undergraduate),10 : 1,19,4,"$2,410","$2,410","$2,410","$2,410","$17,030","$17,030","$17,030","$17,030",NaN,NaN,NaN,NaN,NaN,NaN,94,"$819,301","$8,716",0,$0,—,"$8,311","$7,035","$7,472",NaN
1,1,Abilene Christian University,Abilene Christian University,", Abilene, Texas 79699","Type: 4-year, Private not-for-profit",City: Midsize,Yes,"5,292 (3,525 undergraduate)",15 : 1,274,111,"$32,070","$33,330","$34,850","$36,300","$45,980","$48,300","$49,722","$51,887","$22,356",$0,NaN,NaN,NaN,NaN,"3,443","$69,502,352","$20,187","1,826","$19,263,408","$10,550","$27,175","$27,584","$27,883",NaN
2,2,Abraham Baldwin Agricultural College,Abraham Baldwin Agricultural College,"2802 Moore Hwy, Tifton, Georgia 31793-2601","Type: 4-year, primarily associate's, Public",Town: Distant,Yes,"3,927 (all undergraduate)",17 : 1,150,51,NaN,NaN,NaN,NaN,"$15,025","$15,121","$15,345","$15,479",NaN,NaN,NaN,NaN,NaN,NaN,"3,275","$16,222,202","$4,953","1,500","$8,519,194","$5,679","$8,022","$7,687","$6,823",NaN
3,3,Academy for Nursing and Health Occupations,Academy for Nursing and Health Occupations,"5154 Okeechobee Blvd Ste 201, West Palm Beach,...","Type: 2-year, Private not-for-profit",Suburb: Large,No,552 (all undergraduate),27 : 1,16,10,"$37,000","$37,000","$39,300","$39,300","$86,448","$87,000","$94,025","$97,577",NaN,NaN,NaN,NaN,NaN,NaN,245,"$1,577,912","$6,440",259,"$3,156,855","$12,189","$18,861","$24,009","$20,603",NaN
4,4,Academy of Interactive Entertainment,Academy of Interactive Entertainment,"537 Cajundome Boulevard #211, Lafayette, Louis...","Type: 2-year, Private not-for-profit",City: Midsize,No,88 (all undergraduate),26 : 1,3,1,"$17,500","$18,000","$21,000","$21,750","$34,580","$31,113","$34,113","$36,522",NaN,NaN,NaN,NaN,NaN,NaN,51,"$357,260","$7,005",49,"$407,134","$8,309","$24,970","$23,807","$24,516",NaN
5,5,Adams State University,Adams State University,"208 Edgemont Blvd, Alamosa, Colorado 81101","Type: 4-year, Public",Town: Remote,Yes,"3,114 (1,955 undergraduate)",15 : 1,103,81,NaN,NaN,NaN,NaN,"$22,266","$22,895","$22,916","$22,916",tuition,fees,NaN,NaN,NaN,NaN,"1,363","$11,947,354","$8,765",847,"$5,792,666","$6,839","$12,989","$13,314","$13,383",NaN
6,6,Adelphi University,Adelphi University,"South Ave, Garden City, New York 11530-0701","Type: 4-year, Private not-for-profit",Suburb: Large,Yes,"7,991 (5,360 undergraduate)",11 : 1,361,755,"$35,740","$37,170","$38,740","$40,300","$53,688","$56,358","$58,430","$61,636","$41,920","$1,500",NaN,NaN,NaN,NaN,"4,976","$94,918,781","$19,075","3,209","$22,354,805","$6,966","$27,925","$28,406","$28,130",NaN
7,7,Adrian College,Adrian College,"110 S Madison St, Adrian, Michigan 49221-2575","Type: 4-year, Private not-for-profit",Town: Distant,Yes,"1,828 (1,779 undergraduate)",13 : 1,97,97,"$34,890","$36,010","$37,087","$37,809","$50,706","$52,074","$53,481","$54,829","$18,600",$975,NaN,NaN,NaN,NaN,"1,765","$48,096,035","$27,250","1,397","$6,041,428","$4,325","$26,930","$26,710","$24,

In [34]:
df_tuition = df[df["Tuition_Fee_2016-2017"].isnull()][["CHE_name","CNES_name"]]
df_tuition_scrape = pd.DataFrame(columns = ["CHE_name","CNES_name","tuition_fee_2016_2017_In-state","tuition_fee_2017_2018_In-state","tuition_fee_2019_2020_In-state",
                                            "tuition_fee_2016_2017_Out-state","tuition_fee_2017_2018_Out-state","tuition_fee_2018_2019_Out-state","tuition_fee_2019_2020_Out-state"])
df_tuition_scrape

,CHE_name,CNES_name,tuition_fee_2016_2017_In-state,tuition_fee_2017_2018_In-state,tuition_fee_2019_2020_In-state,tuition_fee_2016_2017_Out-state,tuition_fee_2017_2018_Out-state,tuition_fee_2018_2019_Out-state,tuition_fee_2019_2020_Out-state


In [35]:
#Cleaning Tuition
path = '/Users/Petelindan/Downloads/chromedriver_win32/chromedriver'
chrome_options = Options()
chrome_options.add_argument(" — incognito")
browser = webdriver.Chrome(path, options=chrome_options)

#Open the data source using Chrome 
start_url='https://nces.ed.gov/collegenavigator/'
browser.get(start_url)

for school in df_tuition["CNES_name"]:
    try:
    #For each school, search for the name and navigate to the result page
    #Clearing previous selection and input
        keys = school
        browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName").clear()
        location = Select(browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_ucMapMain_lstState"))
        location.deselect_all()

        #input search keywords
        search_tab = browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName")
        search_tab.send_keys(keys)
#       location.select_by_value(loc)
        browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_btnSearch").click()

        #Result selection page

        newURL = browser.current_url
        browser.get(newURL)
    #         browser.find_element_by_xpath("//a[@id='ctl00_cphCollegeNavBody_ucResultsMain_aSortName']").click()
        browser.find_element_by_xpath("//tr[@class = 'resultsW']/td[2]/a[@href]").click()
    #Extracting data
        browser.find_element_by_xpath("//a[contains(text(),'Expand All')]").click()
    #In-State
        tuition_fee_2016_2017_In_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'In-state')]/following-sibling::td[1]").text
        tuition_fee_2017_2018_In_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'In-state')]/following-sibling::td[2]").text
        tuition_fee_2018_2019_In_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'In-state')]/following-sibling::td[3]").text
        tuition_fee_2019_2020_In_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'In-state')]/following-sibling::td[4]").text
        #Out-State
        tuition_fee_2016_2017_Out_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'Out-of-state')]/following-sibling::td[1]").text
        tuition_fee_2017_2018_Out_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'Out-of-state')]/following-sibling::td[2]").text
        tuition_fee_2018_2019_Out_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'Out-of-state')]/following-sibling::td[3]").text
        tuition_fee_2019_2020_Out_state = browser.find_element_by_xpath("//tr[@class='level1indent']//td[contains(text(),'Out-of-state')]/following-sibling::td[4]").text
        school_data = {"CNES_name":school,
                       "tuition_fee_2016_2017_In_state":tuition_fee_2016_2017_In_state,"tuition_fee_2017_2018_In_state":tuition_fee_2017_2018_In_state, "tuition_fee_2018_2019_In_state": tuition_fee_2018_2019_In_state,
                       "tuition_fee_2019_2020_In_state":tuition_fee_2019_2020_In_state,"tuition_fee_2016_2017_Out_state":tuition_fee_2016_2017_Out_state,"tuition_fee_2017_2018_Out_state":tuition_fee_2017_2018_Out_state,
                      " tuition_fee_2018_2019_Out_state": tuition_fee_2018_2019_Out_state,"tuition_fee_2019_2020_Out_state":tuition_fee_2019_2020_Out_state}
        df_tuition_scrape = df_tuition_scrape.append(school_data,ignore_index = True)
    except:
        pass

In [62]:
df_grad = df[df["AVG_Tuition-Grad-2019-2020"] == "tuition"][["CHE_name","CNES_name"]]
df_grad_scrape = pd.DataFrame(columns =["CNES_name","avg_tuition_grad_2019_2020_Instate","avg_fee_grad_2019_2020_Instate","avg_tuition_grad_2019_2020_OutState","avg_fee_grad_2019_2020_Outstate"])
df_grad_scrape

,CNES_name,avg_tuition_grad_2019_2020_Instate,avg_fee_grad_2019_2020_Instate,avg_tuition_grad_2019_2020_OutState,avg_fee_grad_2019_2020_Outstate


In [63]:
#Cleaning AVG Tuition Grad
path = '/Users/Petelindan/Downloads/chromedriver_win32/chromedriver'
chrome_options = Options()
chrome_options.add_argument(" — incognito")
browser = webdriver.Chrome(path, options=chrome_options)

#Open the data source using Chrome 
start_url='https://nces.ed.gov/collegenavigator/'
browser.get(start_url)

for school in df_grad["CNES_name"]:
    try:
    #For each school, search for the name and navigate to the result page
    #Clearing previous selection and input
        keys = school
        browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName").clear()
        location = Select(browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_ucMapMain_lstState"))
        location.deselect_all()

        #input search keywords
        search_tab = browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_txtName")
        search_tab.send_keys(keys)
    #         location.select_by_value(loc)
        browser.find_element_by_id("ctl00_cphCollegeNavBody_ucSearchMain_btnSearch").click()

        #Result selection page

        newURL = browser.current_url
        browser.get(newURL)
    #         browser.find_element_by_xpath("//a[@id='ctl00_cphCollegeNavBody_ucResultsMain_aSortName']").click()
        browser.find_element_by_xpath("//tr[@class = 'resultsW']/td[2]/a[@href]").click()
    #Extracting data
        browser.find_element_by_xpath("//a[contains(text(),'Expand All')]").click()
        avg_tuition_grad_2019_2020_Instate = browser.find_element_by_xpath("//td[contains(text(),'In-state tuition')]/following-sibling::td[1]").text
        avg_fee_grad_2019_2020_Instate = browser.find_element_by_xpath("//td[contains(text(),'In-state fees')]/following-sibling::td[1]").text
        avg_tuition_grad_2019_2020_OutState = browser.find_element_by_xpath("//td[contains(text(),'Out-of-state tuition')]/following-sibling::td[1]").text    
        avg_fee_grad_2019_2020_Outstate = browser.find_element_by_xpath("//td[contains(text(),'Out-of-state fees')]/following-sibling::td[1]").text

        school_data1 = {"CNES_name":school,
                      "avg_tuition_grad_2019_2020_Instate":avg_tuition_grad_2019_2020_Instate,"avg_fee_grad_2019_2020_Instate":avg_fee_grad_2019_2020_Instate,
                      "avg_tuition_grad_2019_2020_OutState":avg_tuition_grad_2019_2020_OutState,"avg_fee_grad_2019_2020_Outstate":avg_fee_grad_2019_2020_Outstate}
        df_grad_scrape = df_grad_scrape.append(school_data1,ignore_index=True)
    except:
        pass

In [67]:
df_tuition_scrape.to_csv("tuition_fix.csv")

In [68]:
df_grad_scrape.head()

,CNES_name,avg_tuition_grad_2019_2020_Instate,avg_fee_grad_2019_2020_Instate,avg_tuition_grad_2019_2020_OutState,avg_fee_grad_2019_2020_Outstate
0,Adams State University,"$4,590","$2,721","$7,830","$2,721"
1,Alabama State University,"$7,416","$2,740","$14,832","$2,740"
2,Albany State University,"$3,438","$1,870","$13,716","$1,870"
3,Angelo State University,"$4,059","$2,800","$11,655","$2,800"
4,Appalachian State University,"$4,839","$3,137","$18,271","$3,137"
